In [2]:
from typing import *

import torch 
import numpy as np 
import shutil 
import json 
import zipfile 
import io 
import pytorch_lightning as pl 
from scipy.sparse import csc_matrix 
from pathlib import Path 
import torch.nn.functional as F
from torchmetrics.functional import accuracy, precision, recall 
from pytorch_tabnet.tab_network import TabNet
import copy
import warnings
import pandas as pd 
from torchmetrics.functional import *
import torchmetrics 

from torchmetrics.functional.classification.stat_scores import _stat_scores_update, _stat_scores
from sklearn.metrics import classification_report

# from scsims.data import *
# from scsims.lightning_train import *

In [3]:
mouse_labels = pd.read_csv('../../scanvi_benchmark/data/benchmark/mouse_labels.csv')
human_labels = pd.read_csv('../../scanvi_benchmark/data/benchmark/human_labels.csv')

/var/folders/pd/jsjcl0fn7w57s5mfr34b20pm0000gn/T/ipykernel_23365/804854649.py:1: DtypeWarning: Columns (21) have mixed types. Specify dtype option on import or set low_memory=False.
  mouse_labels = pd.read_csv('../../scanvi_benchmark/data/benchmark/mouse_labels.csv')


In [8]:
retina = pd.read_csv('../data/retina/retina_labels_numeric.csv')
retina['class_label'].nunique()

13

In [4]:
mouse_labels['subclass_label'].nunique(), human_labels['subclass_label'].nunique()

(42, 19)

Let's clean the cell type labels for human data since there are some NaN values

In [3]:
fixed_human = human_labels.dropna(subset=['subclass_label'])
fixed_human.index.name = 'cell'
fixed_human = fixed_human.reset_index(drop=False)

fixed_human.to_csv('../data/benchmark/human_labels_clean.csv', index=False)

In [4]:
human_labels = pd.read_csv('../data/benchmark/human_labels_clean.csv')

for label in human_labels['region_label'].unique():
    df = human_labels[human_labels['region_label'] == label]
    
    df.to_csv(f'../data/benchmark/human_labels/human_{label}_labels_clean.csv', index=False)

In [5]:
human_labels

cell          sample_name   exp_component_name specimen_type  \
0          1  F2S4_160113_027_B01  LS-15005h_S02_E1-50       nucleus   
1          2  F2S4_160113_027_C01  LS-15005h_S03_E1-50       nucleus   
2          3  F2S4_160113_027_D01  LS-15005h_S04_E1-50       nucleus   
3          4  F2S4_160113_027_E01  LS-15005h_S05_E1-50       nucleus   
4          5  F2S4_160113_027_F01  LS-15005h_S06_E1-50       nucleus   
...      ...                  ...                  ...           ...   
47427  49412  F2S4_190227_100_C01  SM-GE4QU_S187_E1-50       nucleus   
47428  49413  F2S4_190227_100_E01  SM-GE4QU_S189_E1-50       nucleus   
47429  49414  F2S4_190227_100_F01  SM-GE4QU_S190_E1-50       nucleus   
47430  49415  F2S4_190227_100_G01  SM-GE4QU_S191_E1-50       nucleus   
47431  49416  F2S4_190227_100_H01  SM-GE4QU_S192_E1-50       nucleus   

      cluster_color  cluster_order            cluster_label class_color  \
0           #E170FE           32.0        Inh L2-5 VIP TOX2     #0066FF   
1           #8E5864            2.0       Inh L1 LAMP5 GGT8P     #0066FF   
2           #8B5862            1.0        Inh L1 LAMP5 NDNF     #0066FF   
3           #CF6EC9           34.0    Inh L1-3 VIP ZNF322P1     #0066FF   
4           #E693CE           35.0         Inh L3 VIP CBLN1     #0066FF   
...             ...            ...                      ...         ...   
47427       #312E27          111.0  Astro L1-6 FGFR3 ETNPPL     #17994B   
47428       #BFC124           75.0  Exc L6 THEMIS LINC00343     #AFCC3D   
47429       #8B5862            1.0        Inh L1 LAMP5 NDNF     #0066FF   
47430       #71AF9A          116.0        Oligo L4-6 OPALIN     #17994B   
47431       #71AF9A          116.0        Oligo L4-6 OPALIN     #17994B   

       class_order    class_label  ... cell_type_alt_alias_order  \
0              4.0      GABAergic  ...                      32.0   
1              4.0      GABAergic  ...                       2.0   
2              4.0      GABAergic  ...                       1.0   
3              4.0      GABAergic  ...                      34.0   
4              4.0      GABAergic  ...                      35.0   
...            ...            ...  ...                       ...   
47427          3.0   Non-neuronal  ...                     111.0   
47428          2.0  Glutamatergic  ...                      75.0   
47429          4.0      GABAergic  ...                       1.0   
47430          3.0   Non-neuronal  ...                     116.0   
47431          3.0   Non-neuronal  ...                     116.0   

       cell_type_alt_alias_label cell_type_designation_color  \
0                            NaN                     #E170FE   
1                            NaN                     #8E5864   
2                            NaN                     #8B5862   
3                            NaN                     #CF6EC9   
4                            NaN                     #E693CE   
...                          ...                         ...   
47427                        NaN                     #312E27   
47428                        NaN                     #BFC124   
47429                        NaN                     #8B5862   
47430                        NaN                     #71AF9A   
47431                        NaN                     #71AF9A   

       cell_type_designation_order  cell_type_designation_label  \
0                             32.0                   Neuron 032   
1                              2.0                   Neuron 002   
2                              1.0                   Neuron 001   
3                             34.0                   Neuron 034   
4                             35.0                   Neuron 035   
...                            ...                          ...   
47427                        111.0               Non-neuron 001   
47428                         75.0                   Neuron 075   
47429                          1.0                   

And again for the mouse labels

In [6]:
fixed_mouse = mouse_labels.dropna(subset=['subclass_label'])
fixed_mouse.index.name = 'cell'
fixed_mouse = fixed_mouse.reset_index(drop=False)

fixed_mouse.to_csv('../data/benchmark/mouse_labels_clean.csv', index=False)

In [10]:
mouse_labels = pd.read_csv('../data/benchmark/mouse_labels_clean.csv')

for label in mouse_labels['region_label'].unique():
    df = mouse_labels[mouse_labels['region_label'] == label]
    
    df.to_csv(f'../data/benchmark/mouse_labels/mouse_{label}_labels_clean.csv', index=False)

/var/folders/pd/jsjcl0fn7w57s5mfr34b20pm0000gn/T/ipykernel_87700/3982835164.py:1: DtypeWarning: Columns (22) have mixed types. Specify dtype option on import or set low_memory=False.
  mouse_labels = pd.read_csv('../data/benchmark/mouse_labels_clean.csv')


Since there are some extra sample in the mouse data, we'll take the intersection of the samples labeled and those in the entire dataset, and write it out to a new file.

In [13]:
# comb = list(set(mouse_data.obs['index'].values).intersection(mouse_labels['sample_name']))
# to_keep = mouse_data.obs[mouse_data.obs['index'].isin(comb)].index.astype(int)

# clipped = an.AnnData(
#     X=mouse_data.X[to_keep, :],
#     obs=mouse_data.obs[mouse_data.obs['index'].isin(comb)],
#     var=mouse_data.var
# )
# clipped.write('../data/benchmark/mouse_clipped.h5ad')

Let's look at which genes intersect between the two datasets

In [15]:
hg = [x.upper() for x in human_data.var.index]
mg = [x.upper() for x in mouse_data.var.index]

len(set(hg).intersection(mg))

16519

In [26]:
max(pd.read_csv('../data/mouse/MouseAdultInhibitoryNeurons_labels.csv')['numeric_class'].unique()) + 1

50

In [11]:
testlabels = pd.read_csv('../data/benchmark/mouse_labels/mouse_ALM_labels_clean.csv')
trainlabels = pd.read_csv('../data/benchmark/human_labels/human_MTG_labels_clean.csv')

In [18]:
testlabels['subclass_label'].unique()

array(['Sst', 'Sst Chodl', 'Sncg', 'Vip', 'L6 CT CTX', 'L4/5 IT CTX',
       'L5 IT CTX', 'L2/3 IT CTX', 'Lamp5', 'L5/6 NP CTX', 'Pvalb',
       'Oligo', 'Astro', 'Micro-PVM', 'Endo', 'L6b CTX', 'CR',
       'L6 IT CTX', 'L5 PT CTX', 'Meis2', 'VLMC', 'SMC-Peri', 'Car3'],
      dtype=object)

In [16]:
trainlabels['subclass_label'].unique()

array(['VIP', 'LAMP5', 'IT', 'PAX6', 'Oligodendrocyte', 'Astrocyte',
       'L5/6 IT Car3', 'L5/6 NP', 'SST', 'L6 CT', 'OPC', 'PVALB', 'L6b',
       'Microglia', 'L5 ET', 'Pericyte', 'Endothelial', 'L4 IT', 'VLMC'],
      dtype=object)

In [ ]:
# testlabels --> trainlabels 

mapping = {
    'Sst': 'SST',
    'Sst Chodl': 'SST',
    'Sncg': 'drop',
    'Vip': 'VIP',
    'L6 CT CTX': 'L6 CT',
    'L4/5 IT CTX': 'L4 IT',
    'L5 IT CTX': 'L5/6 IT Car3',
    'L2/3 IT CTX': 'IT',
    'Lamp5': 'LAMP5',
    'L5/6 NP CTX': 
}

In [14]:
pd.read_csv('../data/benchmark/mouse_labels/mouse_VISp_labels_clean.csv')['subclass_label'].unique()

array(['L5 PT CTX', 'L5 IT CTX', 'L4/5 IT CTX', 'L6 IT CTX', 'L6 CT CTX',
       'L5/6 NP CTX', 'Pvalb', 'Vip', 'L2/3 IT CTX', 'Lamp5', 'Sst',
       'Oligo', 'Sst Chodl', 'Sncg', 'Car3', 'Astro', 'L6b CTX', 'Endo',
       'L2/3 IT PPP', 'SMC-Peri', 'Meis2', 'Micro-PVM', 'VLMC', 'CR',
       'L5/6 IT TPE-ENT', 'DG'], dtype=object)

In [19]:
pd.read_csv('../data/bhaduri/primary_bhaduri_labels.csv')

cell  Type
0            0    16
1            1    16
2            2     4
3            3     4
4            4     4
...        ...   ...
186471  189404     9
186472  189405     9
186473  189406     4
186474  189407     8
186475  189408     4

[186476 rows x 2 columns]

In [23]:
pd.read_csv('../data/bhaduri/primary_labels.tsv', sep='\t')['Subtype'].unique()

array(['oRG', 'Layer VI Occipital', 'Newborn', 'vRG', 'early', 'Mural',
       'IPC/newborn', 'OPC', 'Upper Layer Occipital', 'IPC-new',
       'Cajal Retzius', 'Upper Layer', 'Layer VI Pan-area', 'Outlier',
       'Microglia low quality', 'Low Quality', 'SST-MGE1', 'IPC-div1',
       'PFC', 'Upper Layer PFC', 'Microglia', 'Deep Layer', 'late',
       'Parietal and Temporal', 'IPC-div2', 'Layer IV', 'MGE2',
       'Endothelial', 'oRG/Astrocyte', 'tRG'], dtype=object)